In [31]:
%reload_ext autoreload
%autoreload 2

In [69]:
%load_ext blackcellmagic

In [97]:
import warnings
warnings.filterwarnings('ignore')

# Extract

In [16]:
import pandas as pd
import multiprocessing as mp


In [21]:
import extract

In [23]:
import importlib
importlib.reload(extract)

<module 'extract' from '/home/adarsh/dattada/repos/exploration-notebooks/extract.py'>

## for Training

In [78]:
df = extract.get_examples("2018-01-01","2019-12-31")

/home/adarsh/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1411, "Incorrect datetime value: '2019-6-14 20:03:09' for function str_to_date")
  result = self._query(query)
/home/adarsh/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1411, "Incorrect datetime value: '2019-7-23 15:26:47' for function str_to_date")
  result = self._query(query)
/home/adarsh/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1411, "Incorrect datetime value: '2019-7-23 21:15:04' for function str_to_date")
  result = self._query(query)
/home/adarsh/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1411, "Incorrect datetime value: '2019-7-26 15:50:50' for function str_to_date")
  result = self._query(query)
/home/adarsh/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1411, "Incorrect datetime value: '2019-7-26 19:32:12' for function str_to_date")
  result = self._query(query)
/home/adarsh/anacond

## for Evaluation

In [ ]:
df_eval = extract.get_examples("2020-01-01","2020-03-31")

In [44]:
BV_status_list = ['Bank Validation Uncertain', 'Bank Validation Approved']

In [45]:
df_eval = df_eval[df_eval['final_decision'].isin(BV_status_list)]

# EDA

## check missing values

## outlier detection

## check data distribution

# Preprocess

In [50]:
from preprocess import *

## Useful links. 
https://github.com/pycaret/pycaret/blob/master/Tutorials/Binary%20Classification%20Tutorial%20Level%20Intermediate%20-%20CLF102.ipynb

## Feature engineering

### primary account

In [80]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1

In [81]:
with mp.Pool(processes=NCPU) as pool:
    res_primary_accts = pool.map(get_primary_account, df['BankReportData'])

In [82]:
df['primary_account'] = res_primary_accts

In [83]:
df = df.loc[df['primary_account'].notnull(),:]

### filter loans having transaction days >= 60 in primary account

In [84]:
with mp.Pool(processes=NCPU) as pool:
    txn_days_count = pool.starmap(get_transaction_days_count, zip(df['primary_account'],df['BankReportData']))

In [85]:
df['txn_days_count'] = txn_days_count

In [86]:
df = df.loc[df['txn_days_count'] == True, :]

### Calculate Age

In [88]:
df['Age'] = df.apply(lambda x: calculate_age(x['OriginationDate'],x['DateOfBirth']), axis = 1)

### New or Reloan

In [89]:
df['Reloan'] = df['LoanCount'].apply(lambda x:True if x>1 else False)

### Lead Provider

In [91]:
df = df.loc[df['Campaign'].notnull(),:]

In [92]:
lead_provider_list = [
    "MarketBullet",
    "StopNGo",
    "Nimbus",
    "EPCVIP",
    "PingBid",
    "LeapThry",
    "Acquir",
    "RoundSky",
    "Zero",
    "LeadPie",
    "ITMedia",
    "LeadsMarket",
]

In [93]:

df['LeadProvider'] = df['Campaign'].str.extract("(" + "|".join(lead_provider_list) +")",flags = re.IGNORECASE)

In [94]:
df['LeadProvider'] = df['LeadProvider'].fillna('Freedom')

### lender vars

#### in case the lender vars are to be generated for funded loans between 2018-01-01 to 2019-12-31 do not run the below cell, instead download from s3 ( look for "download lender vars from s3" markdown)

In [101]:
df_lender_vars = pd.DataFrame()
with mp.Pool(processes=NCPU) as pool:
    df_lender_vars_temp = pool.starmap(create_lender_vars, zip(df['LoanId'],df['BankReportData'],df['ReportTimeAdded'],df['primary_account']))
df_lender_vars=pd.concat(df_lender_vars_temp,ignore_index=True)

In [98]:
df_lender_vars.reset_index(drop=True,inplace=True)

#### download lender vars from s3

In [ ]:
import boto3

In [ ]:
#provide access keys if needed
s3 = boto3.client('s3')

In [ ]:
s3.download_file('predicon-bucket', 'lender_vars.csv', 'FILE_NAME')

In [ ]:
df_lender_vars = pd.read_csv('lender_vars.csv')

In [99]:
df= pd.merge(df,df_lender_vars,how='left',on='LoanId')

### Number of days positive and negative

In [113]:
with mp.Pool(processes = NCPU) as pool:
    df_gc_vars_temp = pool.starmap(create_gc_vars, zip(df['LoanId'], df['BankReportData'], df['primary_account']))
df_gc_vars = pd.concat(df_gc_vars_temp, ignore_index = True)

### Imputation

### Pycaret preprocessing

In [ ]:
clf = setup(data = df_new, train_size = .99, target = 'IsFirstDefault', session_id = 769,
            normalize = True,
            combine_rare_levels = True,
            transformation = True,
            ignore_low_variance = True, 
            remove_multicollinearity = True,
            bin_numeric_features = ['Age'],
            group_features = [['LenderAmountCred', 'LenderAmountCred30'], ['LenderCountCred', 'LenderCountCred30']],
            feature_interaction = True, feature_ratio = True,
            feature_selection = True,
            polynomial_features = True,
            remove_outliers = True)

# Train

## useful links
https://docs.databricks.com/_static/notebooks/mlflow/mlflow-quick-start-deployment-aws.html

https://towardsdatascience.com/deploying-models-to-production-with-mlflow-and-amazon-sagemaker-d21f67909198

https://www.h2o.ai/blog/a-deep-dive-into-h2os-automl/

## specify features

In [ ]:
features_drop = ['LoanCount',
'OriginationDate',             
'BankReportData',                   
'ReportTimeAdded',                  
'Campaign',
'primary_account',
'txn_days_count', 'DateOfBirth',]

In [ ]:
df_train = df_loans.drop(columns=features_drop,axis=1)

### H20

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(max_mem_size='16G')

In [ ]:
df_h20_train =  h2o.H2OFrame(df_train)

In [ ]:
y = "IsFirstDefault" 
x = df_h20_train.columns
x.remove(y)
x.remove('LoanId')

In [ ]:
aml = H2OAutoML(max_runtime_secs=120, seed=1)
aml.train(x=x, y=y, training_frame=df_h20_train)

In [ ]:
lb = aml.leaderboard
lb.head()

### pycaret

In [ ]:
compare_models()

In [ ]:
tuned_rf = tune_model('rf', optimize = 'F1')

# Predict

### primary account

In [ ]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    res_primary_accts = pool.map(get_primary_account, df_loan_eval['BankReportData'])

In [ ]:
df_loan_eval['primary_account'] = res_primary_accts

In [ ]:
df_loan_eval = df_loan_eval.loc[df_loan_eval['primary_account'].notnull(),:]

### filter loans having transaction days >= 60 in primary account

In [ ]:
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    txn_days_count = pool.starmap(get_transaction_days_count, zip(df_loan_eval['primary_account'],df_loan_eval['BankReportData']))

In [ ]:
df_loan_eval['txn_days_count'] = txn_days_count

In [ ]:
df_loan_eval = df_loan_eval.loc[df_loan_eval['txn_days_count']==True,:]

### calculate age

In [ ]:
df_loan_eval['Age'] = df_loan_eval.apply(lambda x: calculate_age(x['OriginationDate'],x['DateOfBirth']), axis = 1)

### lead provider

In [ ]:
df_loan_eval = df_loan_eval.loc[df_loan_eval['Campaign'].notnull(),:]

In [ ]:
df_loan_eval['LeadProvider'] = df_loan_eval['Campaign'].str.extract("(" + "|".join(lead_provider_list) +")",flags = re.IGNORECASE)

In [ ]:
df_loan_eval['LeadProvider']=df_loan_eval['LeadProvider'].fillna('Freedom')

### new or reloan

In [ ]:
df_loan_eval['Reloan'] = df_loan_eval['LoanCount'].apply(lambda x:True if x>1 else False)

### lender vars

In [ ]:
df_lender_vars_eval = pd.DataFrame()
NCPU = mp.cpu_count() - 2 if mp.cpu_count() > 2 else 1
with mp.Pool(processes=NCPU) as pool:
    df_lender_vars_temp = pool.starmap(create_lender_vars, zip(df_loan_eval['LoanId'],df_loan_eval['BankReportData'],df_loan_eval['ReportTimeAdded'],df_loan_eval['primary_account']))
df_lender_vars_eval=pd.concat(df_lender_vars_temp,ignore_index=True)

In [ ]:
df_loan_eval = pd.merge(df_loan_eval,df_lender_vars_eval,on='LoanId',how='left')

### esign variables

In [ ]:
df_loan_eval= pd.merge(df_loan_eval,df_esign_eval,on='LoanId',how='left')

### run prediction

In [ ]:
df_loan_predict = df_loan_eval.drop(columns=features_drop,axis=1)

In [ ]:
h2o_eval = h2o.H2OFrame(df_loan_predict)

In [ ]:
pred = aml.leader.predict(h2o_eval)
pred.head()

In [ ]:
#convert to pandas dataframe
df_predictions = h2o.as_list(pred)

In [ ]:
df_loan_eval.reset_index(drop=True,inplace=True)

In [ ]:
df_predictions['target'] = df_loan_eval['IsFirstDefault']  

In [ ]:
df_predictions = df_predictions.rename(columns={'True':'prob'})

In [ ]:
df_predictions = df_predictions[['target','prob']]

In [ ]:
df_predictions['target'].value_counts(normalize = True)

# Evaluate

## get BV uncertain and BV Approved loans for model evaluation

In [14]:
username_bank_app = 'bankreview'
password_bank_app = 'Freedom!23'
host_bank_app = '192.168.4.115'
port_bank_app = 3306
db_bank_app = 'bankreviewdb'

In [17]:
bank_app_conn = pymysql.connect(host=host_bank_app,
                                port=port_bank_app,
                                db=db_bank_app,
                                user=username_bank_app,
                                password=password_bank_app)

In [ ]:
query_evaluation_loans = '''select loan_id, 
                                final_decision,
                                reasons_for_decision,
                                entered_date
                                
                            from loan 
                            where campaign like '%Production%'
                            and STR_TO_DATE(entered_date ,'%m/%d/%Y') >= STR_TO_DATE('01/01/2020','%m/%d/%Y')
                            and STR_TO_DATE(entered_date ,'%m/%d/%Y') < STR_TO_DATE('04/01/2020','%m/%d/%Y')
                            and final_decision in ('Bank Validation Uncertain','Bank Validation Approved') '''

In [ ]:
df_eval_loans = pd.read_sql_query(query_evaluation_loans, con = bank_app_conn)

## get funded and mature loans for the same period

In [ ]:
query_funded_mature_loans = ''' select LoanId, 
                                IsFirstDefault
                        from view_FCL_Loan
                        where OriginationDate >= '2020-01-01' 
                        and OriginationDate <= '2020-03-31'
                        and IsFirstDefault IS NOT NULL
                        and MerchantId IN (15, 18)
                        
                     '''

In [ ]:
df_funded_mature_loans = pd.read_sql_query(query_funded_mature_loans,con = iloans_conn)

In [ ]:
df_funded_mature_loans['LoanId'] = df_funded_mature_loans['LoanId'].astype(int).astype(str)

In [ ]:
df_eval = pd.merge(df_funded_mature_loans,df_eval_loans,how = 'inner',left_on = 'LoanId',right_on = 'loan_id')

In [ ]:
df_eval.info()

In [ ]:
loan_id_list = list(df_eval['LoanId'])

### compute KS

In [ ]:
import numpy as np

In [ ]:
def get_KS(df_pred):
    """
    Returns KS given scores
    Parameters:
    df_pred (pandas df): DataFrame containing target variable and model score
    
    Returns:
    float: KS value
    """
    df_scores = df_pred.sort_values(by='prob')
    total_good = (df_scores['target'] == False).sum()
    total_bad = (df_scores['target'] == True).sum()
    df_scores['cum_good_perc'] = (df_scores['target'] == False).cumsum()/total_good
    df_scores['cum_bad_perc'] = (df_scores['target'] == True).cumsum()/total_bad
    df_scores['cum_diff'] = np.abs((df_scores['cum_good_perc'] - df_scores['cum_bad_perc']))
    return df_scores['cum_diff'].max()

In [ ]:
get_KS()

### quantiling

In [ ]:
def quantile_table(df_pred,n = 10):
    """
    Returns a quantile table given model scores (default is decile)
    
    Parameters:
    df_pred (pandas df): DataFrame containing target variable and model score
    
    Returns:
    pandas DataFrame: Pandas dataframe containing quantiles
    
    """
    df_scores = df_pred.sort_values(by='prob')
    df_scores['decile'],score_bin = pd.qcut(df_scores['prob'],10,labels=[1,2,3,4,5,6,7,8,9,10],retbins = True)
    df_scores['target'] = df_scores['target'].astype(int)
    df_scores_deciles = df_scores.groupby('decile',as_index=False).agg({'prob':['count','min','max','mean'],'target':'sum'})
    df_scores_deciles.columns = ['decile','count','min_score','max_score','mean_score','bad_count']
    df_scores_deciles['perc_bad'] = (df_scores_deciles['bad_count']/df_scores_deciles['count']) * 100
    return df_scores_deciles,score_bin

In [ ]:
quantile_table, score_bins = quantile_table(df_predictions)

In [ ]:
quantile_table

### get bins for quantile assignment

In [ ]:
score_bins

In [ ]:
score_bins = np.concatenate(([-np.inf], score_bins, [np.inf]))

### upload to sagemaker

In [ ]:
import mlflow

In [ ]:
import mlflow.h2o as mh2o

In [ ]:
import mlflow.sagemaker as mfs

In [ ]:
mh2o.save_model(aml.leader,path="path/to/trained/model")

In [ ]:
region = "us-east-1"
arn = "arn:aws:iam::757719720041:role/Sagemaker"
appname = "h20-mlflow-deploy"
modeluri = "path/to/saved/model" 
image_url = "757719720041.dkr.ecr.us-east-1.amazonaws.com/freedom-pyfunc:latest"

In [ ]:
mfs.deploy(app_name=appname, model_path=modeluri, instance_type='ml.t2.medium',region_name=region, mode="create",execution_role_arn=arn,image_url=image_url)

In [ ]:
import boto3

def check_status(app_name):
    sage_client = boto3.client('sagemaker', region_name="us-east-1")
    endpoint_description = sage_client.describe_endpoint(EndpointName=app_name)
    endpoint_status = endpoint_description["EndpointStatus"]
    return endpoint_status

In [ ]:
check_status(appname)